In [ ]:
import py4dgeo
import numpy as np
from scipy.spatial import KDTree as ScipyKDTree
from sklearn.neighbors import KDTree as SklearnKDTree

In [ ]:
print("Reading data with numpy from xyz format")
%time data = np.genfromtxt('ahk_2017_small.xyz', delimiter=' ')
print(f"Total number of points in point cloud: {data.shape[0]}")

In [ ]:
print("Transferring the point cloud to C++ PCL data structure")
%time pc = py4dgeo.PCLPointCloud(data)

In [ ]:
print("Benchmarking PCL KDTree")
%time pc.build_tree(py4dgeo.SearchStrategy.kdtree)
%time kd_indices, kd_distances = pc.radius_search(np.array([652876.546, 5189246.239, 2432.163]), 1)

In [ ]:
print("Benchmarking PCL Bruteforce search")
%time pc.build_tree(py4dgeo.SearchStrategy.bruteforce)
%time bf_indices, bf_distances = pc.radius_search(np.array([652876.546, 5189246.239, 2432.163]), 1)

In [ ]:
print("Benchmarking PCL Octree")
%time pc.build_tree(py4dgeo.SearchStrategy.octree)
%time ot_indices, ot_distances = pc.radius_search(np.array([652876.546, 5189246.239, 2432.163]), 1)

In [ ]:
print("Reference implementation: Scipy - pure Python")
%time tree = ScipyKDTree(data)
%time scipy_indices = tree.query_ball_point(np.array([652876.546, 5189246.239, 2432.163]), 1)

In [ ]:
print("Reference implementation: SKLearn - Cython")
%time tree = SklearnKDTree(data)
%time sklearn_indices = tree.query_radius(np.array([[652876.546, 5189246.239, 2432.163]]), 1)

In [ ]:
There is still a functional problem with PCL:

In [ ]:
kd_indices.shape

In [ ]:
bf_indices.shape

In [ ]:
ot_indices.shape

In [ ]:
len(scipy_indices)

In [ ]:
sklearn_indices[0].shape